# Data Profiling
# 0. Set up

In [158]:
# import libraries
import pandas as pd
import seaborn as sns
from cryptography.fernet import Fernet

# load data
df = pd.read_csv('input/dirty-loan-data.csv')
df.shape

(50100, 60)

In [159]:
df.head(5)

,Unnamed: 0,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,50000.0,D,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
1,1,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,50000.0,C,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0
2,2,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,50000.0,D,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
3,3,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,50000.0,A,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
4,4,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,50000.0,A,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0


# 1. Create Data Profile

In [160]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title='Loan Dataset Profiling Report', minimal=True)
profile.to_file(output_file='output/profile/loan_data_profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 1.1. Analyze Integrity
## 1.1.1. Duplicates
To identify duplicates the first step is to remove the first column, which simply contains a row index. If we do not remove this column, the profiling will identify no rows as duplicates.

In [161]:
# drop first column
df = df.drop(columns='Unnamed: 0')

Looking at the output generated by the profiling, the first obvious problem is that the ID variables (**id_pk**, **id_member**) do not seem to be unique. This might be due to duplicated entries in the dataset. We can check this by filtering for all entries containing duplicated 'member_id' and order by 'member_id'.

In [162]:
# filter for all entries containing duplicated 'member_id' order by member_id
df[df.duplicated(subset='member_id', keep=False)].sort_values(by='member_id')

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
50023,435307,520078,7000.0,7000.0,7000.000000,36,4.96,242.52,D,D2,...,NaN,Jul,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
23,435307,520078,7000.0,7000.0,7000.000000,36,4.96,242.52,D,D2,...,NaN,Jul,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
50011,526784,681457,11200.0,11200.0,11164.159885,36,3.23,378.62,C,C1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
11,526784,681457,11200.0,11200.0,11164.159885,36,3.23,378.62,C,C1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
50,558201,718601,7200.0,7200.0,7150.000000,60,7.56,181.12,E,E4,...,NaN,Feb,2015.0,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,67325141,72126853,28800.0,28800.0,28800.000000,60,0.64,621.03,B,B4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,292439.0,36100.0
61,67367721,72179492,3000.0,3000.0,3000.000000,36,7.57,107.82,D,D4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,101900.0,15300.0
50061,67367721,72179492,3000.0,3000.0,3000.000000,36,7.57,107.82,D,D4,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,101900.0,15300.0
50034,67828497,72687218,5000.0,5000.0,5000.000000,36,-4.68,150.58,A,A1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,45434.0,59700.0


Additionally we can also check for rows where all variables are identical. This will return a boolean series where True indicates that the row is a duplicate of a previous row. We can then sum the number of True values to get the number of duplicates.

In [163]:
# show number of duplicates where all rows are equal
df.duplicated().sum()

95

Looking at the output we can see that we indeed have 95 duplicated cases (here the duplicates are identified by rows that contain identical values for all variables). However, we have 200 rows with duplicated member IDs (Here we identify duplicates based on members IDs only). This means that we have some cases where the member ID is duplicated, but the other variables are not identical. This is a problem, as it indicates that we have some inconsistencies in the data. Now let's start by removing the rows where all variables are identical.

In [164]:
# remove duplicates
df = df.drop_duplicates()
df.shape

(50005, 59)

Now we can re-check if we still have duplicates in the member_id variable. As we can see in the output below this is indeed the case. The mistake seems to lie in the variable **installment** where we have differing values for the same case. This is likely a data error, as the variable **installment** should contain the monthly payment owed by the borrower. This means that the value 50000 is likely a mistake, as it is unlikely that a borrower would have to pay 50000 per month. We should therefore remove duplicated cases where installment have a very high value.

In [165]:
# filter for all entries containing duplicated 'member_id' order by member_id and installment
df[df.duplicated(subset='member_id', keep=False)].sort_values(by=['member_id', 'installment'])

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
50004,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,508.06,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0
4,4536779,5779043,16000.0,16000.0,15950.0,36,-1.10,50000.00,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0
50002,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,489.45,D,D2,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
2,18935279,21167968,20000.0,20000.0,20000.0,60,6.29,50000.00,D,D2,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,35408.0,28300.0
50003,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,456.54,A,A1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
3,27580978,30084088,15000.0,15000.0,15000.0,36,-3.97,50000.00,A,A1,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,146372.0,81600.0
50000,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,503.50,D,D1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
0,55978502,59640239,14400.0,14400.0,14400.0,36,5.61,50000.00,D,D1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,15278.0,15000.0
50001,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,818.82,C,C1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0
1,56141977,59803813,24550.0,24550.0,24550.0,36,2.29,50000.00,C,C1,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,163368.0,41400.0


In [166]:
# show row for member id 5779043
df[df['member_id'] == 5779043]

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
4,4536779,5779043,16000.0,16000.0,15950.0,36,-1.1,50000.00,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0
50004,4536779,5779043,16000.0,16000.0,15950.0,36,-1.1,508.06,A,A5,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,355820.0,45600.0


## 1.1.2. Inconsistencies
Another problem that becomes apparent when looking at the data is the discrepancy between the variables **grade** and **subgrade**. The **subgrade** variable is a subset of the **grade** variable. We can check this by looking at the unique values for both variables. However, we can see that the **subgrade** variable contains the nonsensical value *ALPHA_CENTAURI*. We need to address this issue by, first removing this value.

In [167]:
# show unique values for grade and subgrade
(df['grade'].unique(), df['sub_grade'].unique())

(array(['D', 'C', 'A', 'B', 'E', 'F', 'G'], dtype=object),
 array(['D1', 'C1', 'D2', 'A1', 'A5', 'D3', 'B5', 'A4', 'E1', 'C4', 'E2',
        'B2', 'C2', 'E5', 'C3', 'ALPHA_CENTAURI', 'A2', 'C5', 'D5', 'D4',
        'E3', 'B1', 'E4', 'B4', 'A3', 'F2', 'F4', 'F3', 'F1', 'G1', 'F5',
        'G2', 'G3', 'G4', 'G5'], dtype=object))

In [168]:
# replace ALPHA_CENTAURI from sub_grade with None
df['sub_grade'] = df['sub_grade'].replace('ALPHA_CENTAURI', None)

# show unique values for grade and subgrade
df['sub_grade'].unique()

array(['D1', 'C1', 'D2', 'A1', 'A5', 'D3', 'B5', 'A4', 'E1', 'C4', 'E2',
       'B2', 'C2', 'E5', 'C3', None, 'A2', 'C5', 'D5', 'D4', 'E3', 'B1',
       'E4', 'B4', 'A3', 'F2', 'F4', 'F3', 'F1', 'G1', 'F5', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [169]:
# show rows for which sub_grade is NULL
df[df['sub_grade'].isnull()]

,id_pk,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_year,last_credit_pull_month,last_credit_pull_year,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
26,23943598,26316514,12000.0,12000.0,12000.0,36,0.99,392.81,B,None,...,2016.0,Jan,2016.0,1.0,1.0,INDIVIDUAL,0.0,88.0,182767.0,23000.0
54,56052266,59714104,14000.0,14000.0,14000.0,60,-0.01,297.39,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,169022.0,40200.0
79,63518237,67861038,10000.0,10000.0,10000.0,36,-0.01,322.63,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,406066.0,15500.0
96,50655756,54035485,15400.0,15400.0,15400.0,36,-0.01,496.85,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,43947.0,33200.0
102,48676289,51925085,21000.0,21000.0,21000.0,60,-0.01,446.09,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,25175.0,30980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49870,66380227,71104940,10000.0,10000.0,10000.0,36,-0.24,321.55,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,292127.0,8500.0
49922,3370575,4223512,6400.0,6400.0,6400.0,36,2.12,212.94,B,None,...,NaN,Jun,2015.0,0.0,1.0,INDIVIDUAL,0.0,0.0,50077.0,20694.0
49984,18294310,20447017,19500.0,19500.0,19500.0,60,0.99,423.89,B,None,...,NaN,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,56624.0,67700.0
49987,60952305,64994090,12000.0,12000.0,12000.0,36,-0.01,387.15,B,None,...,2016.0,Jan,2016.0,0.0,1.0,INDIVIDUAL,0.0,0.0,43481.0,43300.0


Should we apply a cluster analysis to impute values for subgrade?

* Negative Interest Rates?
* Emp length in num
* annual_inc seems to have outliers

# 1.2. Analyze Completeness
## 1.2.1. Missing Values

# 1.3. Analyze Accuracy
## 1.3.1. Data Types and Formats
Another problem identified in the data are that at the moment dates are split into years and month. This becomes first visible when looking at the variables **issue_year** and **issue_month** as an example. Since we only have observations for 2015, this means that the **issue_year** variable by itself does not provide any useful information (no variation) in future analytical applications. Only in combination with the month, does the year provide useful information.

In [170]:
# show distribution of issue_year
sns.countplot(data=df, x='issue_year')

<Axes: xlabel='issue_year', ylabel='count'>

Contrary to **issue_year** the variable **issue_month** does show some variation, as shown in the plot below.

In [171]:
# show distribution of issue_month
sns.countplot(data=df, x='issue_month')

<Axes: xlabel='issue_month', ylabel='count'>

However, we also see that *October* was misspelled as *Octxyz*. We can fix this by replacing the misspelled value with the correct one.

In [172]:
# replace misspelled value
df['issue_month'] = df['issue_month'].replace('Octxyz', 'Oct')
df['issue_month'].unique()

array(['Jul', 'Aug', 'Jun', 'Oct', 'Nov', 'May', 'Mar', 'Dec', 'Jan',
       'Sep', 'Feb', 'Apr'], dtype=object)

To remedy the fact that issue_year contains no variation, but we still would like to know the year, we can create a new variable that combines the year and month into a single variable. This will allow us to keep the information about the year, but also the month. The same transformation can be applied to the other time variables in the dataset (earliest_cr_line_month, earliest_cr_line_year, last_pymnt_month, last_pymnt_year, next_pymnt_month, next_pymnt_year, last_credit_pull_month, last_credit_pull_year).

In [173]:
# create new variable issue_date with data type datetime
df['issue_date'] = pd.to_datetime(df['issue_year'].astype(str) + '-' + df['issue_month'], format='%Y-%b')

# drop issue_year and issue_month
df = df.drop(columns=['issue_year', 'issue_month'])

# show first 5 rows
df['issue_date'].head(5)

0   2015-07-01
1   2015-08-01
2   2015-06-01
3   2015-10-01
4   2015-08-01
Name: issue_date, dtype: datetime64[ns]

# 1.4. Text Issues
TODO:
* Harmonise Text

# 1.5. Data Protection
We might want to encrypt some variables that contain sensitive information, such as the url. This is because the url might contain information that could be used to identify the borrower. In the case that we need to be able to convert the data back to its original form, we should use encryption. With encryption are able to use a key to encrypt and decrypt data.

In [174]:
# generate private key
private_key = Fernet.generate_key()

# create fernet object
fernet = Fernet(private_key)

# Ensure that 'url' column is of type string if not convert it
if isinstance(df['url'], str) is False:
    df['url'] = df['url'].astype(str)   

# encrypt url
df['url'] = df['url'].apply(lambda x: fernet.encrypt(x.encode()))
df[['member_id', 'url']].head(5)

,member_id,url
0,59640239,b'gAAAAABl2KaAW46TLCDoVzBvZ5ghvALzsHfanqmqcD0v...
1,59803813,b'gAAAAABl2KaA9CaVIhPKzctjnjhdUVgKg4EsrJmaFQFB...
2,21167968,b'gAAAAABl2KaAuDdFEvg30nFnWzSSg8PKEp6x5qLudYql...
3,30084088,b'gAAAAABl2KaAg-MC3vnMOHziEdYkit8XotWL2s_TNFtP...
4,5779043,b'gAAAAABl2KaAgbuhRYQoEoRM6uHnQ2rFgz4868bToMap...


In [175]:
# decrypt url
df['url'] = df['url'].apply(lambda x: fernet.decrypt(x).decode())
df[['member_id', 'url']].head(5)

,member_id,url
0,59640239,https://www.lendingclub.com/browse/loanDetail....
1,59803813,https://www.lendingclub.com/browse/loanDetail....
2,21167968,https://www.lendingclub.com/browse/loanDetail....
3,30084088,https://www.lendingclub.com/browse/loanDetail....
4,5779043,https://www.lendingclub.com/browse/loanDetail....


If we have to permanently anonymize the data we can replace the url with a hash value.

In [176]:
# encrypt url
df['url'] = df['url'].apply(lambda x: hash(x))
df[['member_id', 'url']].head(5)

,member_id,url
0,59640239,-3907560461067247143
1,59803813,118161253516918363
2,21167968,6273067588285047866
3,30084088,7334547639740313669
4,5779043,4243190465071400926
